# Notebook 01: Datenaufbereitung & Feature Engineering

In diesem Notebook werden die Rohdaten importiert, zusammengeführt, bereinigt, um neue Merkmale ergänzt und das vorbereitete DataFrame als CSV exportiert.

## 1. Setup & Dateneinlese  
- 1.1 Imports & Konstanten  
- 1.2 Einlesen aller Quell‐CSVs/Tabellen  

In [1]:
# 1.1 Installation (einmalig pro Kernel)
%pip install openpyxl --quiet

# 1.2 Imports
import os
import csv
import pandas as pd
import numpy as np

# 1.3 Pfade
RAW_DIR       = os.path.join("..", "data", "raw")
PROCESSED_DIR = os.path.join("..", "data", "processed")

# 1.4 Kontrolle: Rohdateien
print("Rohdateien im Verzeichnis:")
for f in sorted(os.listdir(RAW_DIR)):
    print(" -", f)


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Rohdateien im Verzeichnis:
 - US and Canda States.xlsx
 - car_prices.csv


## 2. Daten einlesen & Parsing-Fix
- `car_prices.csv` laden  
- Spot-Check auf verschobene Zeilen  
- Zeilen mit fehlendem Datum reparieren  
- Datum danach konvertieren  
- `states` für USA+Kanada laden

In [2]:
# 2.1 car_prices.csv einlesen und Datum parsen
cars = pd.read_csv(
    os.path.join(RAW_DIR, "car_prices.csv"),
    parse_dates=["saledate"],
    infer_datetime_format=True,
    low_memory=False
)

# 2.2 Alle Zeilen ohne gültiges Datum entfernen
before = len(cars)
cars = cars.dropna(subset=["saledate"]).reset_index(drop=True)
print(f"Entfernte Zeilen ohne Datum: {before - len(cars)}  – Verbleibend: {len(cars)}")

# 2.3 Staaten-Tabelle einlesen
states = pd.read_excel(
    os.path.join(RAW_DIR, "US and Canda States.xlsx"),
    engine="openpyxl"
)

# 2.4 Überblick
print(f"cars:   {cars.shape[0]}×{cars.shape[1]}")
print(f"states: {states.shape[0]}×{states.shape[1]}")
display(cars.head(), states.head())


/tmp/ipykernel_6587/2502480054.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  cars = pd.read_csv(
/tmp/ipykernel_6587/2502480054.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cars = pd.read_csv(


Entfernte Zeilen ohne Datum: 12  – Verbleibend: 558825
cars:   558825×16
states: 72×4


,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,Thu Jan 29 2015 04:30:00 GMT-0800 (PST)
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)


,StateCode,StateName,Region,AlternateName
0,AL,Alabama,Southeast US,Alabama
1,AK,Alaska,West US,Alaska
2,AS,American Samoa,US Territory,Samoa Americana
3,AZ,Arizona,Southwest US,Arizona
4,AR,Arkansas,Southeast US,Arkansas


## 3. Datentypen & Datumskomponenten
- Datentypen prüfen  
- Numerische Spalten casten & Missing-Value-Check  
- Datumskomponenten als Integer extrahieren  

In [3]:
# 3.1 Sicherstellen, dass 'saledate' ein datetime64 ist
cars["saledate"] = pd.to_datetime(
    cars["saledate"], errors="coerce", utc=True
).dt.tz_convert(None)
print("dtype saledate:", cars["saledate"].dtype)

# 3.2 Datentypen vor Cast
print("Datentypen vor Cast:")
print(cars.dtypes, "\n")

# 3.3 Numerische Spalten casten
num_cols = ["condition", "odometer", "mmr", "sellingprice"]
for c in num_cols:
    cars[c] = pd.to_numeric(cars[c], errors="coerce")

# 3.4 Prüfung: Fehlende Werte nach Cast
miss_num = cars[num_cols].isna().sum().to_frame("n_missing")
miss_num["pct_missing"] = 100 * miss_num["n_missing"] / len(cars)
print("Fehlende Werte nach Numerik-Cast:")
display(miss_num, "\n")

# 3.5 Datumskomponenten als Integer extrahieren
cars["sale_year"]    = cars["saledate"].dt.year.astype("Int64")
cars["sale_month"]   = cars["saledate"].dt.month.astype("Int64")
cars["sale_day"]     = cars["saledate"].dt.day.astype("Int64")
cars["sale_weekday"] = cars["saledate"].dt.weekday.astype("Int64")  # 0=Mo … 6=So

# 3.6 Kontrolle
print("Datentypen nach Cast und Extraktion:")
print(cars[num_cols].dtypes)
print(cars[["sale_year","sale_month","sale_day","sale_weekday"]].dtypes, "\n")
display(cars[["saledate","sale_year","sale_month","sale_day","sale_weekday"]].head())


/tmp/ipykernel_6587/1631332055.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cars["saledate"] = pd.to_datetime(


dtype saledate: datetime64[ns]
Datentypen vor Cast:
year                     int64
make                    object
model                   object
trim                    object
body                    object
transmission            object
vin                     object
state                   object
condition              float64
odometer               float64
color                   object
interior                object
seller                  object
mmr                    float64
sellingprice           float64
saledate        datetime64[ns]
dtype: object 

Fehlende Werte nach Numerik-Cast:


,n_missing,pct_missing
condition,11820,2.115152
odometer,94,0.016821
mmr,26,0.004653
sellingprice,0,0.000000


'\n'

Datentypen nach Cast und Extraktion:
condition       float64
odometer        float64
mmr             float64
sellingprice    float64
dtype: object
sale_year       Int64
sale_month      Int64
sale_day        Int64
sale_weekday    Int64
dtype: object 



,saledate,sale_year,sale_month,sale_day,sale_weekday
0,2014-12-16 04:30:00,2014,12,16,1
1,2014-12-16 04:30:00,2014,12,16,1
2,2015-01-14 20:30:00,2015,1,14,2
3,2015-01-28 20:30:00,2015,1,28,2
4,2014-12-18 04:30:00,2014,12,18,3


## 4. State-Codes bereinigen & Merge
- `state` normalisieren  
- Ungültige Codes → `NaN`  
- Linker Merge mit `states` (nur relevante Spalten)  
- Spalten umbenennen und aufräumen  
- Fehlende Merges zählen

In [4]:
# 4.1 state_clean erzeugen
cars["state_clean"] = cars["state"].str.strip().str.upper()

# 4.2 Ungültige Codes (Länge ≠ 2 ODER nicht in states["StateCode"]) auf NaN setzen
valid_codes = set(states["StateCode"])
mask_bad_len = cars["state_clean"].str.len() != 2
mask_not_in_list = ~cars["state_clean"].isin(valid_codes)
cars.loc[mask_bad_len | mask_not_in_list, "state_clean"] = pd.NA

# 4.3 Merge: nur relevante Spalten aus states übernehmen
states_small = states[["StateCode", "StateName", "Region"]]
cars_states = cars.merge(
    states_small,
    how="left",
    left_on="state_clean",
    right_on="StateCode"
)

# 4.4 Spalten umbenennen und redundante entfernen
cars_states = (
    cars_states
    .rename(columns={
        "StateCode":    "state_code",
        "StateName":    "state_name",
        "Region":       "state_region"
    })
    .drop(columns=["state"])               # ursprüngliche Roh-Spalte state
)

# 4.5 Typen prüfen und anpassen
cars_states["state_clean"]  = cars_states["state_clean"].astype("string")
cars_states["state_code"]   = cars_states["state_code"].astype("category")
cars_states["state_region"] = cars_states["state_region"].astype("category")

# 4.6 Kontrolle: Anzahl der fehlenden Merges
n_unmatched = cars_states["state_code"].isna().sum()
total      = len(cars_states)
print(f"Nicht gematchte state_clean: {n_unmatched} von {total} ({100*n_unmatched/total:.4f} %)")

# 4.7 Kurzer Blick auf Ergebnis
display(cars_states[[
    "state_clean","state_code","state_name","state_region"
]].drop_duplicates().head(10))

Nicht gematchte state_clean: 26 von 558825 (0.0047 %)


,state_clean,state_code,state_name,state_region
0,CA,CA,California,West US
7859,TX,TX,Texas,Southwest US
7860,PA,PA,Pennsylvania,Northeast US
7861,MN,MN,Minnesota,Midwest US
7862,AZ,AZ,Arizona,Southwest US
7863,WI,WI,Wisconsin,Midwest US
7865,TN,TN,Tennessee,Southeast US
7866,MD,MD,Maryland,Northeast US
7870,FL,FL,Florida,Southeast US
7871,NE,NE,Nebraska,Midwest US


#### 4.7 Nicht gematchte State-Codes entfernen
- Alle Zeilen, in denen der Merge fehlgeschlagen ist (`state_code` = NaN), werden gelöscht.

In [5]:
# Anzahl vor dem Drop
before_drop = len(cars_states)

# Drop aller Zeilen ohne gültigen state_code
cars_states = cars_states[cars_states["state_code"].notna()].reset_index(drop=True)

# Anzahl entfernte Zeilen ausgeben
removed = before_drop - len(cars_states)
print(f"Entfernte Zeilen ohne gültigen State-Code: {removed}")
print(f"Verbleibende Datensätze: {len(cars_states)}")

Entfernte Zeilen ohne gültigen State-Code: 26
Verbleibende Datensätze: 558799


## 5. Missing-Value-Report & Deskriptive Statistik
- `info()` für Typ- und Null-Counts  
- `describe()` für numerische und kategoriale Übersicht  
- Fehlende Werte pro Spalte (absolut & Prozent)  
- Unique-Counts für kategoriale Features  

In [6]:
# 5.0 Übersicht: Typen und Non-Null-Counts
print("DataFrame Info:")
cars_states.info()

# 5.1 Deskriptive Statistik (numerisch + kategorial)
print("\nNumerische Statistik:")
display(cars_states.describe())
print("\nKategoriale Statistik:")
display(cars_states.describe(include=["object","category"]))

# 5.2 Fehlende Werte pro Spalte
mv = cars_states.isna().sum().to_frame("n_missing")
mv["pct_missing"] = 100 * mv["n_missing"] / len(cars_states)
print("\nMissing-Value-Report:")
display(mv.sort_values("pct_missing", ascending=False))

# 5.3 Unique-Counts für kategoriale Features
cat_cols = cars_states.select_dtypes(include=["object","category"]).columns
unique_counts = {col: cars_states[col].nunique() for col in cat_cols}
print("\nUnique-Werte pro Kategorie:")
for col, cnt in unique_counts.items():
    print(f" - {col}: {cnt}")

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558799 entries, 0 to 558798
Data columns (total 23 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   year          558799 non-null  int64         
 1   make          548498 non-null  object        
 2   model         548400 non-null  object        
 3   trim          548148 non-null  object        
 4   body          545604 non-null  object        
 5   transmission  493448 non-null  object        
 6   vin           558799 non-null  object        
 7   condition     547005 non-null  float64       
 8   odometer      558705 non-null  float64       
 9   color         558050 non-null  object        
 10  interior      558050 non-null  object        
 11  seller        558799 non-null  object        
 12  mmr           558799 non-null  float64       
 13  sellingprice  558799 non-null  float64       
 14  saledate      558799 non-null  datetime64[ns]
 15  s

,year,condition,odometer,mmr,sellingprice,saledate,sale_year,sale_month,sale_day,sale_weekday
count,558799.000000,547005.000000,558705.000000,558799.000000,558799.000000,558799,558799.0,558799.0,558799.0,558799.0
mean,2010.038633,30.672557,68323.830415,13769.377495,13611.356296,2015-03-06 00:09:10.521385728,2014.903853,3.800801,14.634001,1.472229
min,1982.000000,1.000000,1.000000,25.000000,1.000000,2014-01-01 01:15:00,2014.0,1.0,1.0,0.0
25%,2007.000000,23.000000,28374.000000,7100.000000,6900.000000,2015-01-20 18:00:00,2015.0,1.0,7.0,1.0
50%,2012.000000,35.000000,52257.000000,12250.000000,12100.000000,2015-02-12 19:20:00,2015.0,2.0,16.0,1.0
75%,2013.000000,42.000000,99114.000000,18300.000000,18200.000000,2015-05-21 19:00:00,2015.0,6.0,21.0,2.0
max,2015.000000,49.000000,999999.000000,182000.000000,230000.000000,2015-07-20 19:30:00,2015.0,12.0,31.0,6.0
std,3.966830,13.402872,53398.133653,9679.967174,9749.728196,NaN,0.294793,3.230174,8.588076,1.242273



Kategoriale Statistik:


,make,model,trim,body,transmission,vin,color,interior,seller,state_code,state_name,state_region
count,548498,548400,548148,545604,493448,558799,558050,558050,558799,558799,558799,558799
unique,96,973,1962,86,2,550284,20,17,14260,38,38,7
top,Ford,Altima,Base,Sedan,automatic,wbanv13588cz57827,black,black,nissan-infiniti lt,FL,Florida,Southeast US
freq,93553,19349,55815,199429,475904,5,110969,244320,19693,82945,82945,180779



Missing-Value-Report:


,n_missing,pct_missing
transmission,65351,11.694903
body,13195,2.361314
condition,11794,2.110598
trim,10651,1.906052
model,10399,1.860955
make,10301,1.843418
color,749,0.134037
interior,749,0.134037
odometer,94,0.016822
vin,0,0.000000



Unique-Werte pro Kategorie:
 - make: 96
 - model: 973
 - trim: 1962
 - body: 86
 - transmission: 2
 - vin: 550284
 - color: 20
 - interior: 17
 - seller: 14260
 - state_code: 38
 - state_name: 38
 - state_region: 7


## 6. Erste Datenbereinigung nach Merge
- Entfernen von Zeilen ohne gültiges `saledate`  
- Entfernen von Zeilen ohne gültigen `state_clean`  
- Droppen aller Zeilen mit fehlenden Werten in kritischen Spalten  
- Imputation fehlender Werte in `color` und `interior`  
- Kontrolle, dass keine Missing Values mehr verbleiben  

In [7]:
# 6.1 Zeilen ohne gültiges Datum entfernen
before = len(cars_states)
cars_states = cars_states[cars_states["saledate"].notna()].reset_index(drop=True)
removed_date = before - len(cars_states)
print(f"Entfernte Zeilen ohne gültiges saledate: {removed_date}")

# 6.1 Zeilen ohne gültigen State entfernen
before = len(cars_states)
cars_states = cars_states[cars_states["state_clean"].notna()].reset_index(drop=True)
removed_state = before - len(cars_states)
print(f"Entfernte Zeilen ohne gültigen state_clean: {removed_state}\n")

# 6.2 Kritische Spalten – Zeilen mit Missing droppen
critical = ["transmission", "body", "condition", "trim", "make", "model"]
before = len(cars_states)
cars_states = cars_states.dropna(subset=critical).reset_index(drop=True)
removed_critical = before - len(cars_states)
print(f"Entfernte Zeilen mit fehlenden Werten in {critical}: {removed_critical}")
print(f"Verbleibende Datensätze: {len(cars_states)}\n")

# 6.3 Zeilen mit fehlenden Werten in color oder interior droppen
before = len(cars_states)
cars_states = cars_states.dropna(subset=["color", "interior"]).reset_index(drop=True)
removed = before - len(cars_states)
print(f"Entfernte Zeilen mit fehlenden color oder interior: {removed}")
print(f"Verbleibende Datensätze: {len(cars_states)}\n")

# 6.4 Zeilen ohne odometer droppen
before = len(cars_states)
cars_states = cars_states.dropna(subset=["odometer"]).reset_index(drop=True)
removed = before - len(cars_states)
print(f"Entfernte Zeilen ohne odometer: {removed}")
print(f"Verbleibende Datensätze: {len(cars_states)}\n")

# 6.5 Kontrolle: verbleibende Missing Values
mv = cars_states.isna().sum().to_frame("n_missing")
mv["pct_missing"] = 100 * mv["n_missing"] / len(cars_states)
print("Verbleibende Missing Values nach Drop:")
display(mv[mv["n_missing"] > 0])

Entfernte Zeilen ohne gültiges saledate: 0
Entfernte Zeilen ohne gültigen state_clean: 0

Entfernte Zeilen mit fehlenden Werten in ['transmission', 'body', 'condition', 'trim', 'make', 'model']: 85908
Verbleibende Datensätze: 472891

Entfernte Zeilen mit fehlenden color oder interior: 545
Verbleibende Datensätze: 472346

Entfernte Zeilen ohne odometer: 21
Verbleibende Datensätze: 472325

Verbleibende Missing Values nach Drop:


,n_missing,pct_missing


## 7. Ausreisser entfernen (numerische Features)
- `car_age` berechnen  
- Extremwerte im `sellingprice`, `odometer`, `car_age` und `mmr` beschneiden (1 %–99 %-Quantile)  
- (Optional) nur Fahrzeuge ab einem bestimmten Jahr behalten  

In [8]:
# 7.0 car_age berechnen (falls noch nicht vorhanden)
cars_states["car_age"] = cars_states["sale_year"] - cars_states["year"]

# 7.1 Quantile berechnen
quantiles = {}
for col in ["sellingprice", "odometer", "car_age", "mmr"]:
    q_low, q_high = cars_states[col].quantile([0.01, 0.99])
    quantiles[col] = (q_low, q_high)
    print(f"{col}: 1 %-Quantil = {q_low:.2f}, 99 %-Quantil = {q_high:.2f}")

# 7.2 Ausreisser-Filter anwenden
before = len(cars_states)
mask = pd.Series(True, index=cars_states.index)
for col, (low, high) in quantiles.items():
    mask &= cars_states[col].between(low, high)
cars_states = cars_states[mask].reset_index(drop=True)
after = len(cars_states)

print(f"\nEntfernte Ausreisser: {before - after} Zeilen")
print(f"Verbleibende Datensätze: {after}")

sellingprice: 1 %-Quantil = 500.00, 99 %-Quantil = 44500.00
odometer: 1 %-Quantil = 3343.72, 99 %-Quantil = 221145.28
car_age: 1 %-Quantil = 0.00, 99 %-Quantil = 16.00
mmr: 1 %-Quantil = 800.00, 99 %-Quantil = 44500.00

Entfernte Ausreisser: 20935 Zeilen
Verbleibende Datensätze: 451390


## 8. Kategoriale Reduktion (Top-K Auswahl)
- Für jede wichtige Kategorie nur die häufigsten Werte (z. B. bis 80 % Abdeckung) behalten  
- Alle anderen Labels auf `"Other"` mappen  
- Reduktion auf Top 10 Marken, Top 5 Regionen und gängige Karosserieformen  

In [9]:
# 8.1 Hilfsfunktion: Top-K nach kumulierter Abdeckung ermitteln
def top_k_cover(series, coverage=0.8):
    freqs = series.value_counts(normalize=True)
    cum = freqs.cumsum()
    k = cum.searchsorted(coverage) + 1
    return freqs.index[:k].tolist()

# 8.2 Top 10 Marken ermitteln
make_counts = cars_states["make"].value_counts()
top_makes = make_counts.index[:10].tolist()
print(f"Top 10 Marken: {top_makes}")

# 8.2.1 Zeilen mit anderen Marken entfernen
before = len(cars_states)
cars_states = cars_states[cars_states["make"].isin(top_makes)].reset_index(drop=True)
removed = before - len(cars_states)
print(f"Entfernte Zeilen mit nicht-Top 10-Marken: {removed}")
print(f"Verbleibende Datensätze: {len(cars_states)}")

# 8.3 Top-Regionen (state_region) – z.B. Top 5
top_regions = cars_states["state_region"].value_counts().index[:5].tolist()
print(f"Top Regionen: {top_regions}")

# 8.4 Gängige Karosserieformen (body) – z.B. Top 6
top_bodies = cars_states["body"].value_counts().index[:6].tolist()
print(f"Top Karosserieformen: {top_bodies}")

# 8.5 Kontrolle: Häufigkeiten der Top-K-Werte (+ "Other")
mappings = {
    "make":         top_makes,
    "state_region": top_regions,
    "body":         top_bodies
}

for col, top_list in mappings.items():
    # Temporär auf Object casten
    temp = cars_states[col].astype("object")
    temp = temp.where(temp.isin(top_list), other="Other")
    counts = temp.value_counts()
    
    # Reihenfolge: Top-Werte + "Other"
    keys = top_list + ["Other"]
    # reindex mit fill_value=0, falls "Other" nicht vorkommt
    counts_ordered = counts.reindex(keys, fill_value=0)
    
    print(f"\nSpalte '{col}':")
    print(counts_ordered)

Top 10 Marken: ['Ford', 'Chevrolet', 'Nissan', 'Toyota', 'Dodge', 'Honda', 'Hyundai', 'BMW', 'Kia', 'Chrysler']
Entfernte Zeilen mit nicht-Top 10-Marken: 130612
Verbleibende Datensätze: 320778
Top Regionen: ['Southeast US', 'Midwest US', 'West US', 'Northeast US', 'Southwest US']
Top Karosserieformen: ['Sedan', 'SUV', 'sedan', 'Minivan', 'suv', 'Hatchback']

Spalte 'make':
make
Ford         78370
Chevrolet    51804
Nissan       43007
Toyota       33771
Dodge        26450
Honda        23343
Hyundai      18023
BMW          15902
Kia          15472
Chrysler     14636
Other            0
Name: count, dtype: int64

Spalte 'state_region':
state_region
Southeast US    107604
Midwest US       71021
West US          59307
Northeast US     46412
Southwest US     34998
Other             1436
Name: count, dtype: int64

Spalte 'body':
body
Sedan        124443
SUV           60444
sedan         26572
Minivan       16751
suv           12206
Hatchback     12039
Other         68323
Name: count, dtype: in

#### 8.6 Filter auf alle drei Top-Kategorien
- Nur Zeilen behalten, in denen **alle drei** Bedingungen erfüllt sind:  
  1. `make` ∈ `top_makes`  
  2. `state_region` ∈ `top_regions`  
  3. `body` ∈ `top_bodies`  
- Alle anderen Zeilen werden entfernt  

In [10]:
# 8.6 Strenger Filter: alle drei Bedingungen müssen erfüllt sein
mask_strict = (
    cars_states["make"].isin(top_makes) &
    cars_states["state_region"].isin(top_regions) &
    cars_states["body"].isin(top_bodies)
)

before = len(cars_states)
cars_states = cars_states[mask_strict].reset_index(drop=True)
removed = before - len(cars_states)

print(f"Entfernte Zeilen, die nicht alle drei Top-Kategorien erfüllen: {removed}")
print(f"Verbleibende Datensätze: {len(cars_states)}")

Entfernte Zeilen, die nicht alle drei Top-Kategorien erfüllen: 69496
Verbleibende Datensätze: 251282


## 9. Weitere Datensatz-Verkleinerung
- Filtere den DataFrame anhand zusätzlicher Kriterien, um näher an ~100 000 Zeilen zu kommen  
  1. Nur Top 5 Marken behalten  
  2. Nur Fahrzeuge ab Baujahr 2010  
  3. Engerer Ausreisser-Filter für `sellingprice` (5 %–95 % Quantil)  
  4. (Optional) Pro Marke bis zu N Zeilen extrahieren, um die Zielgrösse exakt zu treffen  

In [11]:
# 9.1 Top 5 Marken definieren
top5_makes = cars_states["make"].value_counts().index[:5].tolist()
print("Top 5 Marken:", top5_makes)

# 9.2 sale_year ≥ 2010
mask = cars_states["sale_year"] >= 2010

# 9.3 Top 5 makes
mask &= cars_states["make"].isin(top5_makes)

# 9.4 Engerer Preis-Quantil-Filter (5 %–95 %)
low, high = cars_states["sellingprice"].quantile([0.05, 0.95])
mask &= cars_states["sellingprice"].between(low, high)

# 9.5 Anwenden und Zwischenstand
before = len(cars_states)
cars_states = cars_states[mask].reset_index(drop=True)
print(f"Nach kombiniertem Filter entfernt: {before - len(cars_states)} Zeilen")
print(f"Verbleibende Datensätze: {len(cars_states)}\n")

# 9.6 Optional: Pro Marke gleich viele Zeilen entnehmen, um exakt ~100 000 zu erreichen
# Ziel pro make: 100000 / 5 = 20000
N_per_make = 20000
dfs = []
for m in top5_makes:
    sub = cars_states[cars_states["make"] == m]
    dfs.append(sub.head(N_per_make))
cars_states = pd.concat(dfs).reset_index(drop=True)
print(f"Nach Sampling pro Marke verbleibend: {len(cars_states)} Zeilen")


Top 5 Marken: ['Ford', 'Nissan', 'Chevrolet', 'Toyota', 'Honda']


Nach kombiniertem Filter entfernt: 89932 Zeilen
Verbleibende Datensätze: 161350

Nach Sampling pro Marke verbleibend: 98129 Zeilen


## 10. Feature Engineering  
Erstellung neuer Features für diverse Aspekte des Datensatzes.

#### 10.1 Regionale Preisstatistiken  
Berechnung von `avg_price_state` und `median_price_state` pro Bundesstaat und Merge ins DataFrame `cars_states`.

In [12]:
# 10.1.1 Preisstatistiken pro Bundesstaat berechnen
state_stats = (
    cars_states
    .groupby("state_code")["sellingprice"]
    .agg(avg_price_state="mean", median_price_state="median")
    .reset_index()
)

# 10.1.2 Merge der Statistik zurück in cars_states
cars_states = cars_states.merge(
    state_stats,
    on="state_code",
    how="left"
)

# 10.1.3 Kontrolle: einige Beispiele anzeigen
print("Beispiele für regionale Preisstatistiken (state_code):")
display(state_stats.head())

# 10.1.4 Verifikation im Haupt-DataFrame
sample_code = state_stats["state_code"].iloc[0]
print(f"Verifizierungsbeispiel für {sample_code}:")
display(
    cars_states
    .loc[cars_states["state_code"] == sample_code, 
         ["state_code", "avg_price_state", "median_price_state"]]
    .head(3)
)

Beispiele für regionale Preisstatistiken (state_code):


/tmp/ipykernel_6587/3404876562.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("state_code")["sellingprice"]


,state_code,avg_price_state,median_price_state
0,AB,NaN,NaN
1,AL,10800.000000,10800.0
2,AZ,10970.367937,10800.0
3,CA,11319.833386,11250.0
4,CO,13404.706491,13400.0


Verifizierungsbeispiel für AB:


,state_code,avg_price_state,median_price_state


#### 10.2 Saison-Feature & Marken-Modell-spezifische Saisonpreise  
Einordnung des Verkaufsmonats in Jahreszeiten und Berechnung des durchschnittlichen Verkaufspreises je Kombination aus `make`, `model` und `season`.

In [13]:
# 10.2.1 Funktion zur Jahreszeit-Zuordnung
def assign_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Fruehling"
    elif month in [6, 7, 8]:
        return "Sommer"
    else:
        return "Herbst"

# 10.2.2 Saison-Feature erstellen
cars_states["season"] = cars_states["sale_month"].astype(int).apply(assign_season)

# 10.2.3 Durchschnittspreise nach Make-Model-Season berechnen
make_model_season_price = (
    cars_states
    .groupby(["make", "model", "season"])["sellingprice"]
    .mean()
    .reset_index(name="mean_price_make_model_season")
)

# 10.2.4 Kontrolle: Beispiel-Ausgabe
print("Beispiel: Durchschnittspreise für einige Make-Model-Season-Kombinationen")
display(make_model_season_price.head(10))

# 10.2.5 Anzahl der Seasons pro Make-Model prüfen
season_counts = (
    make_model_season_price
    .groupby(["make", "model"])["season"]
    .nunique()
    .reset_index(name="season_variants")
)
print("Make-Model-Kombinationen in >1 Season verkauft:", (season_counts["season_variants"] > 1).sum())

Beispiel: Durchschnittspreise für einige Make-Model-Season-Kombinationen


,make,model,season,mean_price_make_model_season
0,Chevrolet,Astro,Winter,4077.777778
1,Chevrolet,Astro Cargo,Winter,3620.000000
2,Chevrolet,Aveo,Fruehling,4150.000000
3,Chevrolet,Aveo,Sommer,3800.000000
4,Chevrolet,Aveo,Winter,4389.164087
5,Chevrolet,Blazer,Fruehling,2600.000000
6,Chevrolet,Blazer,Winter,3034.615385
7,Chevrolet,Camaro,Fruehling,2450.000000
8,Chevrolet,Camaro,Winter,3316.666667
9,Chevrolet,Captiva Sport,Fruehling,15800.000000


Make-Model-Kombinationen in >1 Season verkauft: 82


### 10.3 Text-Mining-Indikatoren  
Erstellung von Binär-Flags (`has_sport`, `has_limited`, `has_lx`, etc.) basierend auf Stichwörtern in den Spalten `trim` und `model`.

In [14]:
# 10.3.1 Schlüsselwörter definieren
keywords = ["Sport", "Limited", "LX", "SE", "Touring", "Premium"]

# 10.3.2 Für jedes Keyword eine Binär-Spalte erzeugen
for kw in keywords:
    col_name = f"has_{kw.lower()}"
    cars_states[col_name] = (
        cars_states["trim"].str.contains(kw, case=False, na=False) |
        cars_states["model"].str.contains(kw, case=False, na=False)
    ).astype(int)

# 10.3.3 Kontrolle: einige Beispiele anzeigen
display(
    cars_states[
        ["trim", "model"] + [f"has_{kw.lower()}" for kw in keywords]
    ].head(10)
)

,trim,model,has_sport,has_limited,has_lx,has_se,has_touring,has_premium
0,SE,Fusion,0,0,0,1,0,0
1,Limited,Escape,0,1,0,0,0,0
2,SEL,Edge,0,0,0,1,0,0
3,SEL,Edge,0,0,0,1,0,0
4,Titanium,Focus,0,0,0,0,0,0
5,Titanium,Focus,0,0,0,0,0,0
6,SE,Focus,0,0,0,1,0,0
7,SES,Fiesta,0,0,0,1,0,0
8,SEL,Focus,0,0,0,1,0,0
9,SE,Fiesta,0,0,0,1,0,0


### 10.4 Nutzungskennzahlen & Farb-Popularität  
Berechnung der durchschnittlichen Jahresfahrleistung (`miles_per_year`) und des Rangs der Lackfarbe (`color_popularity`).

In [15]:
# 10.4.1 Fahrzeugalter berechnen und Nullwerte vermeiden
cars_states["vehicle_age"] = cars_states["sale_year"] - cars_states["year"]
cars_states.loc[cars_states["vehicle_age"] == 0, "vehicle_age"] = 1

# 10.4.2 Durchschnittliche Jahresfahrleistung
cars_states["miles_per_year"] = cars_states["odometer"] / cars_states["vehicle_age"]

# 10.4.3 Rang der Lackfarbe ermitteln
color_counts = cars_states["color"].value_counts()
color_rank   = color_counts.rank(method="dense", ascending=False).astype(int)
cars_states["color_popularity"] = cars_states["color"].map(color_rank)

# 10.4.4 Kontrolle: einige Beispiele anzeigen
display(
    cars_states[
        ["odometer", "vehicle_age", "miles_per_year", 
         "color", "color_popularity"]
    ].head(10)
)

,odometer,vehicle_age,miles_per_year,color,color_popularity
0,5559.0,1,5559.0,white,4
1,45035.0,2,22517.5,gray,3
2,20035.0,2,10017.5,gray,3
3,41115.0,2,20557.5,white,4
4,26747.0,2,13373.5,red,6
5,29499.0,2,14749.5,black,1
6,64750.0,2,32375.0,red,6
7,38159.0,2,19079.5,white,4
8,68306.0,2,34153.0,white,4
9,36404.0,2,18202.0,red,6


## 11. Exklusion des MMR-Features  
In diesem Schritt wird das `mmr`-Feature entfernt, um mögliche Datenlecks zu vermeiden. Das Ergebnis wird in `cars_states_no_mmr` gespeichert.

In [16]:
# 11.1 Temporäres DataFrame ohne 'mmr' erstellen
cars_states_no_mmr = cars_states.drop("mmr", axis=1).copy()

# 11.2 Kontrolle: Shape und Kopf des neuen DataFrames
print(f"Ursprüngliches DataFrame: {cars_states.shape}")
print(f"DataFrame ohne 'mmr': {cars_states_no_mmr.shape}\n")
display(cars_states_no_mmr.head())

Ursprüngliches DataFrame: (98129, 36)
DataFrame ohne 'mmr': (98129, 35)



,year,make,model,trim,body,transmission,vin,condition,odometer,color,...,season,has_sport,has_limited,has_lx,has_se,has_touring,has_premium,vehicle_age,miles_per_year,color_popularity
0,2015,Ford,Fusion,SE,Sedan,automatic,3fa6p0hdxfr145753,2.0,5559.0,white,...,Winter,0,0,0,1,0,0,1,5559.0,4
1,2012,Ford,Escape,Limited,SUV,automatic,1fmcu0eg0ckb55384,35.0,45035.0,gray,...,Winter,0,1,0,0,0,0,2,22517.5,3
2,2012,Ford,Edge,SEL,SUV,automatic,2fmdk3jc5cba41602,46.0,20035.0,gray,...,Winter,0,0,0,1,0,0,2,10017.5,3
3,2012,Ford,Edge,SEL,SUV,automatic,2fmdk4jc4cba12890,46.0,41115.0,white,...,Winter,0,0,0,1,0,0,2,20557.5,4
4,2012,Ford,Focus,Titanium,Hatchback,automatic,1fahp3n21cl330426,3.0,26747.0,red,...,Winter,0,0,0,0,0,0,2,13373.5,6


## 12. Export des bereinigten DataFrames  
Alle Bereinigungen und Feature-Erweiterungen sind abgeschlossen. Das finale DataFrame `cars_states_no_mmr` wird als CSV für nachfolgende Notebooks gespeichert.

In [17]:
import os

# 12.1 Output-Verzeichnis (anpassen falls nötig)
PROCESSED_DIR = "../data/processed"

# 12.2 Kontrolle der DataFrame-Grösse
print("Verbleibende Datensätze:", len(cars_states_no_mmr))

# 12.3 CSV speichern
out_path = os.path.join(PROCESSED_DIR, "cars_states_cleaned.csv")
cars_states_no_mmr.to_csv(out_path, index=False)
print("Zwischenergebnis gespeichert unter:", out_path)

Verbleibende Datensätze: 98129
Zwischenergebnis gespeichert unter: ../data/processed/cars_states_cleaned.csv
